# Frictionless Product Categorization

- **Author:** Jose Vicente Ruiz Cepeda (jr3660)
- **Course:** COMS W 4995 - Deep Learning for Computer Vision
- **Assignment**: Final project

## Context

Explain some of the context of the problem here.

## Code

### Imports

In [2]:
import keras # Keras 1.2.2 assumed.
from keras import optimizers

from keras.models import Model

from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications import VGG16
from keras.applications import VGG19

from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.inception_v3 import preprocess_input

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten

from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

import os
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import confusion_matrix

# Required to avoid errors with the images.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

Using TensorFlow backend.


### Constants

In [3]:
CATEGORIES = {
    0: 'photo',
    1: 'electronics',
    2: 'events',
    3: 'instruments',
    4: 'tools',
    5: 'sports',
    6: 'caravans',
    7: 'others'
}

SUBCATEGORIES = {
    0:  '360',
    1:  'action_cameras',
    2:  'drones',
    3:  'instant_cameras',
    4:  'kits',
    5:  'lenses',
    6:  'lighting',
    7:  'others',
    8:  'photo_cameras',
    9:  'projectors',
    10: 'sound',
    11: 'supports',
    12: 'video_cameras'
}

### Functions

In [4]:
# Modified from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=60)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "%d%%" % int(cm[i, j]*100),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [5]:
def show_sample(X, y, class_names, prediction=-1):
    im = X
    print y
    #y = np.flip(y, axis=0)
    y_label = class_names[y]
    plt.imshow(im)
    if prediction >= 0:
        plt.title("Class = %s, Predict = %s" % (y_label, class_names[prediction]))
    else:
        plt.title("Class = %s" % (y_label))

    plt.axis('on')
    plt.show()

### Part I - Categories

#### Training

In [6]:
train_data_dir = './data/train'
validation_data_dir = './data/validation'
test_data_dir = './data/test'

In [7]:
class Categorizer:
    def __init__(self, classes):
        self.classes = classes
        
    def build_model(self, 
                    ImagenetModel=ResNet50,
                    weights='imagenet',
                    num_dense_layer_units=256,
                    dropout=0.5,
                    multiclass_output=False,
                    verbose=False):
        # Keras models are functions, not classes, so we have to check which one is it like this.
        if ImagenetModel.func_name == 'InceptionV3' or ImagenetModel.func_name == 'Xception':
            self.img_width, self.img_height = 299, 299
        else:
            self.img_width, self.img_height = 224, 224

        # First, let's load the model with ImageNet weights and without the top layer.
        # This will take some time the first time, since the weights have to be downloaded.
        model_weights = weights if weights == 'imagenet' else None
        model = ImagenetModel(
            weights=model_weights,
            include_top=False,
            input_shape=(self.img_width, self.img_height, 3)
        )
        self.bottom_model = model

        # Now, let's create the top layers adapted to our problem. 
        preds = self._build_top_model(
            num_dense_layer_units=num_dense_layer_units,
            dropout=dropout,
            multiclass_output=multiclass_output)

        # Combine both models to get the final one.
        self.model = Model(model.input, preds)

        if verbose:
            self.model.summary()
            
        # If weights is a string different from ImageNet, path to weights is assumed.
        if os.path.exists(weights):
            self.model.load_weights(weights)

        return self
    
    def change_top_model(self,
                         new_classes,
                         num_dense_layer_units=256,
                         dropout=0.5,
                         multiclass_output=False,
                         verbose=False):
        self.classes = new_classes
        model = self.bottom_model
        preds = self._build_top_model(
            num_dense_layer_units=num_dense_layer_units,
            dropout=dropout,
            multiclass_output=multiclass_output)

        self.model = Model(model.input, preds)
        if verbose:
            self.model.summary()

        return self
        
    def _build_top_model(self,
                         num_dense_layer_units=256,
                         dropout=0.5,
                         multiclass_output=False):
        # We use the Functional API of Keras. 
        # https://keras.io/getting-started/functional-api-guide/
        model = self.bottom_model
        x = model.output
        x = Flatten(input_shape=model.output_shape[1:])(x)
        x = Dense(num_dense_layer_units, activation='relu')(x)
        x = Dropout(dropout)(x)
        preds = Dense(
            len(self.classes),
            activation='sigmoid' if multiclass_output else 'softmax')(x)
        self.num_top_layers = 4
        
        return preds
    
    def compile_model(self, optimizer=optimizers.Adagrad(lr=0.001)):
        self.is_compiled = True
        self.optimizer = optimizer
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy']
        )
            
        return self
    
    def _recompile_model(self):
        if self.is_compiled != True:
            raise Error("Model has to be compiled first.")

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=self.optimizer,
            metrics=['accuracy']
        )
    
    def fine_tune(self,
                  train_data_dir,
                  validation_data_dir,
                  batch_size=16,
                  num_only_top_epochs=10,
                  num_whole_model_epochs=40,
                  best_model_path=None,
                  tensorboard_logs_path=None,
                  reduce_learning_rate=True):
        
        # Augmentation configurations for training and validation.
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
        validation_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')

        validation_generator = validation_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        if num_only_top_epochs > 0:
            # Freeze everything except the top layers, before training.
            for layer in self.model.layers[:-self.num_top_layers]:
                layer.trainable = False 

            print "Starting with top layers training..."
            self._fit_generator(
                train_generator,
                validation_generator,
                batch_size,
                num_only_top_epochs,
                best_model_path=best_model_path,
                tensorboard_logs_path=tensorboard_logs_path,
                tensorboard_logs_path_suffix='Top',
                reduce_learning_rate=True)
            print "Top layers training done."
            
        if num_whole_model_epochs > 0:
            # Unfreeze everything and train for some more epochs.
            for layer in self.model.layers:
                layer.trainable = True
            
            print "Starting with whole model training..."
            self._fit_generator(
                train_generator,
                validation_generator,
                batch_size,
                num_whole_model_epochs,
                best_model_path=best_model_path,
                tensorboard_logs_path=tensorboard_logs_path,
                tensorboard_logs_path_suffix='Whole',
                reduce_learning_rate=True)
            print "Whole model training done"
        
        return self
    
    def predict(self, X_batch):
        return self.model.predict(X_batch)
        
    def predict_generator(self, test_data_dir):
        # As explained in https://github.com/fchollet/keras/issues/3896,
        # there is an unresolved issue with predict_generator that causes
        # errors when the number of samples is not divisible by the batch.
        # size. To avoid this problem, let's use batch_size=1, for now.
        batch_size = 1
        
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        
        return self.model.predict_generator(
            test_generator,
            test_generator.n//batch_size)
    
    def predict_classes(self, test_data_dir):
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        for X_batch, Y_batch in train_generator:
            for i in range(len(Y_batch)):
                show_sample(X_batch[i, :, :, :], Y_batch[i])
            break
        
    
    def evaluate(self, test_data_dir, batch_size=32):
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        return self.model.evaluate_generator(
            test_generator,
            test_generator.n//batch_size)
    
    # Private methods.
    def _fit_generator(self,
                       train_generator,
                       validation_generator,
                       batch_size,
                       num_epochs,
                       best_model_path=None,
                       tensorboard_logs_path=None,
                       tensorboard_logs_path_suffix='',
                       reduce_learning_rate=True):
        # Recompile model before training to increase efficiency in
        # case of frozen layers.
        self._recompile_model()
        
        # Save the best model based on validation accuracy.
        if best_model_path:
            model_checkpoint = ModelCheckpoint(
                best_model_path,
                monitor='val_acc',
                save_best_only=True,
                save_weights_only=False,
                period=1
            )
        
        # Log epochs information like loss and accuracy to review it
        # afterwards using TensorBoard.
        if tensorboard_logs_path:
            if tensorboard_logs_path[-1] == '/':
                tensorboard_logs_path = tensorboard_logs_path[:-1]
            tensorboard = TensorBoard(
                log_dir=tensorboard_logs_path+'-'+tensorboard_logs_path_suffix+'/')
        
        # Reduce learning rate i
        if reduce_learning_rate:
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
        
        self.model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.n//batch_size,
            nb_epoch=num_epochs,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.n//batch_size,
            callbacks=[model_checkpoint, tensorboard, reduce_lr])

#### Testing

## Part II: Subcategories

In [8]:
subcategories_train_data_dir = './subcategories-data/train'
subcategories_validation_data_dir = './subcategories-data/validation'
subcategories_test_data_dir = './subcategories-data/test'

In [9]:
Categorizer(CATEGORIES). \
    build_model(VGG16, weights='best_models/VGG16.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG16-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG16-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/1
368/383 [===========================>..] - ETA: 0s - loss: 7.2776 - acc: 0.0027 

/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


384/383 [==============================] - 12s - loss: 7.2063 - acc: 0.0026 - val_loss: 6.5475 - val_acc: 0.0000e+00
Top layers training done.
Starting with whole model training...
Epoch 1/1
176/383 [============>.................] - ETA: 15s - loss: 6.6375 - acc: 0.0057

/usr/lib/python2.7/dist-packages/PIL/Image.py:857: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


384/383 [==============================] - 28s - loss: 6.7998 - acc: 0.0104 - val_loss: 5.3413 - val_acc: 0.0208
Whole model training done


<__main__.Categorizer instance at 0x7fbeec44ee60>

In [11]:
Categorizer(SUBCATEGORIES). \
    build_model(VGG16, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG16-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG16-subcats-imagenet'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/1
384/383 [==============================] - 10s - loss: 2.5656 - acc: 0.1328 - val_loss: 2.2142 - val_acc: 0.1146
Top layers training done.
Starting with whole model training...
Epoch 1/1
384/383 [==============================] - 27s - loss: 2.8849 - acc: 0.1250 - val_loss: 2.2295 - val_acc: 0.0938
Whole model training done


<__main__.Categorizer instance at 0x7fbe8942dea8>

In [12]:
Categorizer(CATEGORIES). \
    build_model(VGG19, weights='best_models/VGG19.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG19-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG19-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/1
384/383 [==============================] - 12s - loss: 11.3504 - acc: 0.0104 - val_loss: 13.1778 - val_acc: 0.0104
Top layers training done.
Starting with whole model training...
Epoch 1/1
384/383 [==============================] - 32s - loss: 11.0728 - acc: 0.0182 - val_loss: 13.5093 - val_acc: 0.0000e+00
Whole model training done


<__main__.Categorizer instance at 0x7fbe871b76c8>

In [13]:
Categorizer(CATEGORIES). \
    build_model(VGG19, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG19-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG19-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/1
48/47 [==============================] - 3s - loss: 1.0108 - acc: 0.6875 - val_loss: 3.2250e-04 - val_acc: 1.0000
Top layers training done.
Starting with whole model training...
Epoch 1/1
48/47 [==============================] - 6s - loss: 0.0865 - acc: 0.9375 - val_loss: 1.1921e-07 - val_acc: 1.0000
Whole model training done


<__main__.Categorizer instance at 0x7fbeec47d1b8>

In [14]:
Categorizer(CATEGORIES). \
    build_model(ResNet50, weights='best_models/ResNet50.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/ResNet50-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/ResNet50-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/1
384/383 [==============================] - 16s - loss: 2.2988 - acc: 0.2135 - val_loss: 2.2479 - val_acc: 0.1354
Top layers training done.
Starting with whole model training...
Epoch 1/1
384/383 [==============================] - 25s - loss: 2.3106 - acc: 0.2318 - val_loss: 4.1637 - val_acc: 0.1354
Whole model training done


<__main__.Categorizer instance at 0x7fbe702b4dd0>

In [15]:
Categorizer(CATEGORIES). \
    build_model(ResNet50, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/ResNet50-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/ResNet50-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/1
48/47 [==============================] - 25s - loss: 1.5234 - acc: 0.6875 - val_loss: 1.3200 - val_acc: 1.0000
Top layers training done.
Starting with whole model training...
Epoch 1/1
48/47 [==============================] - 7s - loss: 0.2312 - acc: 1.0000 - val_loss: 0.1668 - val_acc: 1.0000
Whole model training done


<__main__.Categorizer instance at 0x7fbe702b4e60>

In [16]:
Categorizer(CATEGORIES). \
    build_model(Xception, weights='best_models/Xception.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/Xception-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/Xception-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/1
384/383 [==============================] - 43s - loss: 2.1470 - acc: 0.2708 - val_loss: 2.0793 - val_acc: 0.3333
Top layers training done.
Starting with whole model training...
Epoch 1/1
384/383 [==============================] - 49s - loss: 2.2092 - acc: 0.2812 - val_loss: 2.0577 - val_acc: 0.3646
Whole model training done


<__main__.Categorizer instance at 0x7fbe08095e18>

In [17]:
Categorizer(CATEGORIES). \
    build_model(Xception, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/Xception-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/Xception-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/1
48/47 [==============================] - 38s - loss: 0.9838 - acc: 0.5833 - val_loss: 1.7881e-07 - val_acc: 1.0000
Top layers training done.
Starting with whole model training...
Epoch 1/1
48/47 [==============================] - 14s - loss: 0.1130 - acc: 0.9583 - val_loss: 1.1921e-07 - val_acc: 1.0000
Whole model training done


<__main__.Categorizer instance at 0x7fbeed992290>

In [18]:
Categorizer(CATEGORIES). \
    build_model(InceptionV3, weights='best_models/InceptionV3.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/InceptionV3-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/InceptionV3-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/1
384/383 [==============================] - 114s - loss: 2.9726 - acc: 0.0156 - val_loss: 2.6039 - val_acc: 0.0104
Top layers training done.
Starting with whole model training...
Epoch 1/1
384/383 [==============================] - 38s - loss: 3.8170 - acc: 0.0443 - val_loss: 4.1145 - val_acc: 0.1354
Whole model training done


<__main__.Categorizer instance at 0x7fbdc389d830>

In [19]:
Categorizer(CATEGORIES). \
    build_model(InceptionV3, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/InceptionV3-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/InceptionV3-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/1
48/47 [==============================] - 153s - loss: 1.9215 - acc: 0.5625 - val_loss: 0.0827 - val_acc: 1.0000
Top layers training done.
Starting with whole model training...
Epoch 1/1
48/47 [==============================] - 16s - loss: 1.7190 - acc: 0.8542 - val_loss: 15.1107 - val_acc: 0.0625
Whole model training done


<__main__.Categorizer instance at 0x7fbdc389d050>

# Sandbox